In [1]:
DATA_PATH = '../../data/Classification/processed/2_preprocessed_df.pkl'


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [3]:
df = pd.read_pickle(DATA_PATH)

In [4]:
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,image
0,HAM_0000118,ISIC_0027419,2.0,3.0,80.0,0.0,9.0,../../data/Classification/raw\all\ISIC_0027419...,"[[[189, 152, 194], [192, 156, 198], [191, 154,..."
1,HAM_0000118,ISIC_0025030,2.0,3.0,80.0,0.0,9.0,../../data/Classification/raw\all\ISIC_0025030...,"[[[24, 13, 22], [24, 14, 22], [24, 14, 26], [2..."
2,HAM_0002730,ISIC_0026769,2.0,3.0,80.0,0.0,9.0,../../data/Classification/raw\all\ISIC_0026769...,"[[[186, 127, 135], [189, 133, 145], [192, 135,..."
3,HAM_0002730,ISIC_0025661,2.0,3.0,80.0,0.0,9.0,../../data/Classification/raw\all\ISIC_0025661...,"[[[24, 11, 17], [24, 11, 20], [30, 15, 25], [4..."
4,HAM_0001466,ISIC_0031633,2.0,3.0,75.0,0.0,11.0,../../data/Classification/raw\all\ISIC_0031633...,"[[[131, 88, 110], [142, 97, 120], [152, 107, 1..."
...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,0.0,3.0,40.0,0.0,4.0,../../data/Classification/raw\all\ISIC_0033084...,"[[[143, 125, 134], [140, 122, 131], [140, 122,..."
10011,HAM_0002867,ISIC_0033550,0.0,3.0,40.0,0.0,4.0,../../data/Classification/raw\all\ISIC_0033550...,"[[[5, 6, 3], [6, 7, 4], [7, 8, 5], [8, 8, 6], ..."
10012,HAM_0002867,ISIC_0033536,0.0,3.0,40.0,0.0,4.0,../../data/Classification/raw\all\ISIC_0033536...,"[[[109, 99, 102], [122, 111, 112], [132, 121, ..."
10013,HAM_0000239,ISIC_0032854,0.0,3.0,80.0,0.0,5.0,../../data/Classification/raw\all\ISIC_0032854...,"[[[157, 121, 143], [160, 125, 145], [155, 114,..."


In [5]:
features=df.drop(columns=['dx'],axis=1)
target=df['dx']

In [6]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.25,random_state=666)

In [7]:
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [8]:
# Perform one-hot encoding on the labels
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [9]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 999)
x_train = x_train.reshape(x_train.shape[0], *(100, 125, 3))
x_test = x_test.reshape(x_test.shape[0], *(100, 125, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(100, 125, 3))

In [10]:
print(x_train.shape)
print(x_test.shape)

(6759, 100, 125, 3)
(2504, 100, 125, 3)


In [11]:
x_train = x_train.reshape(6759,125*100*3)
x_test = x_test.reshape(2504,125*100*3)
print(x_train.shape)
print(x_test.shape)

(6759, 37500)
(2504, 37500)


In [12]:
# define the keras model
model = Sequential()

model.add(Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu', input_dim = 37500))
model.add(Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00075,
                                    beta_1 = 0.9,
                                    beta_2 = 0.999,
                                    epsilon = 1e-8)

# compile the keras model
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# fit the keras model on the dataset
history = model.fit(x_train, y_train, batch_size = 10, epochs = 50)

accuracy = model.evaluate(x_test, y_test, verbose=1)[1]
print("Test: accuracy = ",accuracy*100,"%")

Epoch 1/50
676/676 [==============================] - 15s 12ms/step - loss: 0.9894 - accuracy: 0.6707
Epoch 2/50
676/676 [==============================] - 8s 12ms/step - loss: 0.8976 - accuracy: 0.6880
Epoch 3/50
676/676 [==============================] - 8s 12ms/step - loss: 0.8502 - accuracy: 0.7022
Epoch 4/50
676/676 [==============================] - 8s 12ms/step - loss: 0.8213 - accuracy: 0.7119
Epoch 5/50
676/676 [==============================] - 8s 12ms/step - loss: 0.7858 - accuracy: 0.7196
Epoch 6/50
676/676 [==============================] - 8s 12ms/step - loss: 0.7605 - accuracy: 0.7273
Epoch 7/50
676/676 [==============================] - 8s 12ms/step - loss: 0.7279 - accuracy: 0.7381
Epoch 8/50
676/676 [==============================] - 8s 12ms/step - loss: 0.7028 - accuracy: 0.7463
Epoch 9/50
676/676 [==============================] - 8s 13ms/step - loss: 0.6781 - accuracy: 0.7514
Epoch 10/50
676/676 [==============================] - 8s 13ms/step - loss: 0.6574 - accur

In [17]:
X = df.drop(['lesion_id','image_id','path','image','dx'],axis=1)
y = df['dx']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [18]:
clf = svm.SVC(kernel='linear') 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [19]:
print("Accuracy: %.2f%%" % (metrics.accuracy_score(y_test, y_pred) * 100.0))

Accuracy: 67.01%
